In [116]:
import pandas as pd
import numpy as np
from langdetect import detect
from tqdm import tqdm

pd.set_option('display.max_columns', None)

In [117]:
# pip install yake
# pip install langdetect
# pip install googletrans

In [118]:
# read in fundamentals of ~5500 companies
fox = pd.read_csv('traderfox/traderfox.csv')
fox['lang_dsc'] = fox['description'].apply(lambda x: detect(str(x)))
print(fox.columns)
len(fox)

Index(['name', 'description', '2021_Dividende', '2020_Dividende',
       '2019_Dividende', '2018_Dividende', '2017_Dividende', '2016_Dividende',
       '2015_Dividende', '2014_Dividende', '2013_Dividende', '2021_EBIT',
       '2020_EBIT', '2019_EBIT', '2018_EBIT', '2017_EBIT', '2016_EBIT',
       '2015_EBIT', '2014_EBIT', '2013_EBIT', '2021_EPS', '2020_EPS',
       '2019_EPS', '2018_EPS', '2017_EPS', '2016_EPS', '2015_EPS', '2014_EPS',
       '2013_EPS', '2021_Gewinn', '2020_Gewinn', '2019_Gewinn', '2018_Gewinn',
       '2017_Gewinn', '2016_Gewinn', '2015_Gewinn', '2014_Gewinn',
       '2013_Gewinn', '2021_KGV', '2020_KGV', '2019_KGV', '2018_KGV',
       '2017_KGV', '2016_KGV', '2015_KGV', '2014_KGV', '2013_KGV', '2021_KUV',
       '2020_KUV', '2019_KUV', '2018_KUV', '2017_KUV', '2016_KUV', '2015_KUV',
       '2014_KUV', '2013_KUV', '2021_Umsatz', '2020_Umsatz', '2019_Umsatz',
       '2018_Umsatz', '2017_Umsatz', '2016_Umsatz', '2015_Umsatz',
       '2014_Umsatz', '2013_Umsatz', '2022_

5419

In [119]:
# read in tickers of the companies (seperately scraped)
tickers = pd.read_csv('traderfox/traderfox_ticker.csv')
tickers.columns = ['name2', 'ticker']
print(tickers.columns)
len(tickers)

Index(['name2', 'ticker'], dtype='object')


5419

In [120]:
# read in further sectors of the companies (seperately scraped)
sectors = pd.read_csv('stockslist.csv')
print(sectors.columns)
len(sectors)

Index(['Symbol', 'Company Name', 'Industry', 'Market Cap'], dtype='object')


6040

In [121]:
# as the dfs of fundamentals and tickers are matching we simply concatenate
data = pd.concat([tickers[['ticker']], fox], axis=1)

In [122]:
# read in further sectors
fox_sectors = pd.read_csv('traderfox/traderfox_sectors.csv')
fox_sectors = pd.concat([fox_sectors, pd.read_csv('traderfox/traderfox_sectors_2.csv')], axis=0)
fox_sectors.columns = ['name3', 'sector']
print(fox_sectors.columns)
len(fox_sectors)

Index(['name3', 'sector'], dtype='object')


4870

In [123]:
# as there are name duplicates (because of the way scraping was done (could be improved))
fox_sectors = fox_sectors.drop_duplicates(subset=['name3'])
print(fox_sectors.columns)
len(fox_sectors)

Index(['name3', 'sector'], dtype='object')


4650

In [124]:
# 0 - 396           DAX
# 397 - 4600        NASDAQ & NYSE
# 4601 - 5194       EURO STOXX
# 5195 - end        NIKKEI

In [125]:
# merging data by tickers and names
merged = pd.merge(data[397:4601], sectors, how='left', left_on='ticker', right_on='Symbol')
merged = pd.concat([data[:397], merged], axis=0)
merged = pd.concat([merged, data[4601:]], axis=0)
merged = pd.merge(merged, fox_sectors, how='left', left_on='name', right_on='name3')

#reordering columns
first_cols = ['ticker', 'name', 'description'] + list(sectors.columns) + list(fox_sectors.columns)
cols = first_cols + [col for col in merged.columns if col not in first_cols]
merged = merged[cols]

In [126]:
merged

,ticker,name,description,Symbol,Company Name,Industry,Market Cap,name3,sector,2021_Dividende,2020_Dividende,2019_Dividende,2018_Dividende,2017_Dividende,2016_Dividende,2015_Dividende,2014_Dividende,2013_Dividende,2021_EBIT,2020_EBIT,2019_EBIT,2018_EBIT,2017_EBIT,2016_EBIT,2015_EBIT,2014_EBIT,2013_EBIT,2021_EPS,2020_EPS,2019_EPS,2018_EPS,2017_EPS,2016_EPS,2015_EPS,2014_EPS,2013_EPS,2021_Gewinn,2020_Gewinn,2019_Gewinn,2018_Gewinn,2017_Gewinn,2016_Gewinn,2015_Gewinn,2014_Gewinn,2013_Gewinn,2021_KGV,2020_KGV,2019_KGV,2018_KGV,2017_KGV,2016_KGV,2015_KGV,2014_KGV,2013_KGV,2021_KUV,2020_KUV,2019_KUV,2018_KUV,2017_KUV,2016_KUV,2015_KUV,2014_KUV,2013_KUV,2021_Umsatz,2020_Umsatz,2019_Umsatz,2018_Umsatz,2017_Umsatz,2016_Umsatz,2015_Umsatz,2014_Umsatz,2013_Umsatz,2022_Dividende,2022_EBIT,2022_EPS,2022_Gewinn,2022_KGV,2022_KUV,2022_Umsatz,lang_dsc
0,1U1,11-ag,Die 1&1 AG ist ein börsennotierter Telekommuni...,NaN,NaN,NaN,NaN,11-ag,Telekommunikation / Telekomdienstleister,0.0039,0.0039,0.0039,0.0039,0.1241,0.1396,0.1358,0.1319,0.1241,535120000.0,312590000.0,522440000.0,562580000.0,431290000.0,55560000.0,65800000.0,72610000.0,167670000.0,2.10,1.25,2.12,2.30,1.91,0.48,0.85,1.03,3.24,370020000.0,219590000.0,373620000.0,406030000.0,258930000.0,26430000.0,45800000.0,50080000.0,155770000.0,6.14,10.35,6.08,5.61,6.75,26.85,15.16,12.51,3.98,0.58,0.60,0.62,0.62,0.81,0.99,1.12,2.37,2.13,3.910000e+09,3.790000e+09,3.670000e+09,3.660000e+09,2.810000e+09,710010000.0,629550000.0,289650000.0,290470000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,de
1,TGT,11-88-0-solutions-ag,Die 11 88 0 Solutions AG (ehemals telegate AG)...,NaN,NaN,NaN,NaN,11-88-0-solutions-ag,Telekommunikation / Telekomdienstleister,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.3718,-640000.0,-2950000.0,-2360000.0,-3010000.0,-9870000.0,-14500000.0,-10410000.0,-9110000.0,-2790000.0,0.04,-0.11,-0.16,-0.16,-0.49,-0.76,-0.47,-0.35,-0.07,1040000.0,-2310000.0,-3180000.0,-3110000.0,-9630000.0,-14670000.0,-9190000.0,-6770000.0,-1380000.0,25.24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.47,0.52,0.47,0.48,0.50,0.46,0.38,0.33,0.28,5.653000e+07,5.080000e+07,4.767000e+07,4.292000e+07,4.127000e+07,44710000.0,53540000.0,62260000.0,72330000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,de
2,2GB,2g-energy-ag,2G Energy AG gehört zu den etablierten Unterne...,NaN,NaN,NaN,NaN,2g-energy-ag,-,0.0053,0.0048,0.0048,0.0048,0.0044,0.0042,0.0039,0.0039,0.0039,17720000.0,16250000.0,15120000.0,11190000.0,6960000.0,3480000.0,4490000.0,10940000.0,2800000.0,0.70,0.68,0.58,0.42,0.28,0.10,0.16,0.37,0.05,12610000.0,11970000.0,10310000.0,7520000.0,5010000.0,1800000.0,2780000.0,6490000.0,890000.0,33.57,34.91,40.55,55.66,83.69,231.37,149.37,64.48,472.00,1.59,1.69,1.77,1.99,2.21,2.40,2.74,2.24,3.32,2.663500e+08,2.467300e+08,2.364000e+08,2.097800e+08,1.894000e+08,174300000.0,152880000.0,186610000.0,126130000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,de
3,UUU,3u-holding-ag,Die 3U HOLDING AG mit Sitz in Marburg wurde 19...,NaN,NaN,NaN,NaN,3u-holding-ag,Telekommunikation / Telekomdienstleister,0.0125,0.0125,0.0100,0.0075,0.0050,0.0025,NaN,NaN,NaN,6290000.0,5260000.0,4640000.0,1830000.0,1900000.0,100000.0,-290000.0,-3330000.0,-5060000.0,0.08,0.10,0.12,0.06,0.03,0.02,-0.02,-0.09,-0.12,2920000.0,3270000.0,4090000.0,1930000.0,1070000.0,640000.0,-820000.0,-3240000.0,-4120000.0,48.07,42.00,32.44,68.79,133.00,199.50,NaN,NaN,NaN,2.52,2.31,2.65,2.76,2.82,3.02,2.74,2.74,3.49,5.594000e+07,6.105000e+07,5.145000e+07,4.797000e+07,4.689000e+07,43740000.0,48240000.0,49240000.0,39710000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,de
4,HRPK,7c-solarparken-ag,Die 7C Solarparken AG zählt zu den mittelgroße...,NaN,NaN,NaN,NaN,7c-solarparken-ag,Alternative Energieerzeugung / Solarenergie,0.0249,0.0249,0.0248,0.0247,0.0225,NaN,NaN,NaN,NaN,11210000.0,7170000.0,8640000.0,8180000.0,7460000.0,6220000.0,6310000.0,7820000.0,-3330000.0,0.14,0.08,0.14,0.12,0.14,0.11,0.16,0.28,-0.36,9860000.0,5250000.0,7880000.0,5890000.0,6020000.0,4710000.0,5500000.0,7390000.0,-3040000.0,32.17,52.47,31.71,37.35,31.94,40.44,27.90,15.91,NaN,6.26,5.92,6.31,5.

In [127]:
# create multiindices for fundamentals
first_level = [i for i in range(2013,2023)]
second_level = ['Dividende', 'EBIT', 'EPS', 'Gewinn', 'KGV', 'KUV', 'Umsatz']

multiindex = pd.MultiIndex.from_product([first_level, second_level])

In [128]:
# prepare the df for col-renaming and split up company_info and fundamentals
sel = sorted([col for col in merged.columns if col.startswith('2')])
facts = merged[[col for col in merged.columns if col not in sel]]
merged = merged[sel]
merged.columns = multiindex

In [129]:
# another possible multiindex for company infos
first_level = ['company']
second_level = ['ticker', 'name', 'sector', 'lang_s', 'description', 'lang_d']

# Combine the levels into a MultiIndex
multiindex = pd.MultiIndex.from_product([first_level, second_level])

In [ ]:
# clean facts
facts['name'] = facts['name'].apply(lambda x: x.replace('-', ' '))
facts['Company Name'] = facts['Company Name'].combine_first(facts['name'])
facts['Company Name'] = facts['Company Name'].apply(str.lower)
# facts['Industry'] = facts['Industry'].combine_first(facts['sector']).fillna('-').replace('-', 'unknown')
# facts['Industry'] = facts['Industry'].apply(lambda x: x.replace('/', 'und'))
facts['lang_sec'] = facts['Industry'].apply(lambda x: detect(str(x)))

facts = facts[['ticker', 'Company Name', 'Industry', 'lang_sec', 'description', 'lang_dsc',]].replace('-', np.nan)
facts.columns = second_level

# reconcat facts and funds
df = pd.concat([facts, merged], axis=1)

In [ ]:
# following is to scale company's 

In [131]:
from sklearn.preprocessing import MinMaxScaler

In [132]:
l = [col for col in df.columns if col[1] == 'Umsatz']

umsatz = df[l].T
scaler = MinMaxScaler().fit(umsatz)
umsatz = scaler.transform(umsatz)

x = [(col[0], 'Umsatz_scaled') for col in df.columns if col[1] == 'Umsatz']

df = pd.concat([df, pd.DataFrame(umsatz.T, columns=x)], axis=1)
df

/home/mabioo/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:461: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/home/mabioo/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:462: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


,ticker,name,sector,lang_s,description,lang_d,"(2013, Dividende)","(2013, EBIT)","(2013, EPS)","(2013, Gewinn)","(2013, KGV)","(2013, KUV)","(2013, Umsatz)","(2014, Dividende)","(2014, EBIT)","(2014, EPS)","(2014, Gewinn)","(2014, KGV)","(2014, KUV)","(2014, Umsatz)","(2015, Dividende)","(2015, EBIT)","(2015, EPS)","(2015, Gewinn)","(2015, KGV)","(2015, KUV)","(2015, Umsatz)","(2016, Dividende)","(2016, EBIT)","(2016, EPS)","(2016, Gewinn)","(2016, KGV)","(2016, KUV)","(2016, Umsatz)","(2017, Dividende)","(2017, EBIT)","(2017, EPS)","(2017, Gewinn)","(2017, KGV)","(2017, KUV)","(2017, Umsatz)","(2018, Dividende)","(2018, EBIT)","(2018, EPS)","(2018, Gewinn)","(2018, KGV)","(2018, KUV)","(2018, Umsatz)","(2019, Dividende)","(2019, EBIT)","(2019, EPS)","(2019, Gewinn)","(2019, KGV)","(2019, KUV)","(2019, Umsatz)","(2020, Dividende)","(2020, EBIT)","(2020, EPS)","(2020, Gewinn)","(2020, KGV)","(2020, KUV)","(2020, Umsatz)","(2021, Dividende)","(2021, EBIT)","(2021, EPS)","(2021, Gewinn)","(2021, KGV)","(2021, KUV)","(2021, Umsatz)","(2022, Dividende)","(2022, EBIT)","(2022, EPS)","(2022, Gewinn)","(2022, KGV)","(2022, KUV)","(2022, Umsatz)","(2013, Umsatz_scaled)","(2014, Umsatz_scaled)","(2015, Umsatz_scaled)","(2016, Umsatz_scaled)","(2017, Umsatz_scaled)","(2018, Umsatz_scaled)","(2019, Umsatz_scaled)","(2020, Umsatz_scaled)","(2021, Umsatz_scaled)","(2022, Umsatz_scaled)"
0,1U1,11 ag,NaN,tl,Die 1&1 AG ist ein börsennotierter Telekommuni...,de,0.1241,167670000.0,3.24,155770000.0,3.98,2.13,290470000.0,0.1319,72610000.0,1.03,50080000.0,12.51,2.37,289650000.0,0.1358,65800000.0,0.85,45800000.0,15.16,1.12,629550000.0,0.1396,55560000.0,0.48,26430000.0,26.85,0.99,710010000.0,0.1241,431290000.0,1.91,258930000.0,6.75,0.81,2.810000e+09,0.0039,562580000.0,2.30,406030000.0,5.61,0.62,3.660000e+09,0.0039,522440000.0,2.12,373620000.0,6.08,0.62,3.670000e+09,0.0039,312590000.0,1.25,219590000.0,10.35,0.60,3.790000e+09,0.0039,535120000.0,2.10,370020000.0,6.14,0.58,3.910000e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000226,0.000000,0.093886,0.116110,0.696162,0.930946,0.933708,0.966854,1.000000,NaN
1,TGT,11 88 0 solutions ag,NaN,tl,Die 11 88 0 Solutions AG (ehemals telegate AG)...,de,0.3718,-2790000.0,-0.07,-1380000.0,NaN,0.28,72330000.0,NaN,-9110000.0,-0.35,-6770000.0,NaN,0.33,62260000.0,NaN,-10410000.0,-0.47,-9190000.0,NaN,0.38,53540000.0,NaN,-14500000.0,-0.76,-14670000.0,NaN,0.46,44710000.0,NaN,-9870000.0,-0.49,-9630000.0,NaN,0.50,4.127000e+07,NaN,-3010000.0,-0.16,-3110000.0,NaN,0.48,4.292000e+07,NaN,-2360000.0,-0.16,-3180000.0,NaN,0.47,4.767000e+07,NaN,-2950000.0,-0.11,-2310000.0,NaN,0.52,5.080000e+07,NaN,-640000.0,0.04,1040000.0,25.24,0.47,5.653000e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.675789,0.395042,0.110753,0.000000,0.053123,0.206053,0.306825,0.491307,NaN
2,2GB,2g energy ag,NaN,tl,2G Energy AG gehört zu den etablierten Unterne...,de,0.0039,2800000.0,0.05,890000.0,472.00,3.32,126130000.0,0.0039,10940000.0,0.37,6490000.0,64.48,2.24,186610000.0,0.0039,4490000.0,0.16,2780000.0,149.37,2.74,152880000.0,0.0042,3480000.0,0.10,1800000.0,231.37,2.40,174300000.0,0.0044,6960000.0,0.28,5010000.0,83.69,2.21,1.894000e+08,0.0048,11190000.0,0.42,7520000.0,55.66,1.99,2.097800e+08,0.0048,15120000.0,0.58,10310000.0,40.55,1.77,2.364000e+08,0.0048,16250000.0,0.68,11970000.0,34.91,1.69,2.467300e+08,0.0053,17720000.0,0.70,12610000.0,33.57,1.59,2.663500e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.431322,0.190772,0.343532,0.451220,0.596563,0.786407,0.860077,1.000000,NaN
3,UUU,3u holding ag,NaN,tl,Die 3U HOLDING AG mit Sitz in Marburg wurde 19...,de,NaN,-5060000.0,-0.12,-4120000.0,NaN,3.49,39710000.0,NaN,-3330000.0,-0.09,-3240000.0,NaN,2.74,49240000.0,NaN,-290000.0,-0.02,-820000.0,NaN,2.74,48240000.0,0.0025,100000.0,0.02,640000.0,199.50,3.02,43740000.0,0.0050,1900000.0,0.03,1070000.0,133.00,2.82,4.689000e+07,0.0075,1830000.0,0.06,1930000.0,68.79,2.76,4.797000e+07,0.0100,4640000.0,0.12,4090000.0,32.44,2.65,5.145000e+07,0.0125,5260000.0,0.10,3270000.0,42.00,2.31,6.105000e+

In [135]:
# filtering data for tableau presentation
murica = df[397:4601]
murica['sector'].value_counts()

Biotechnology                 494
Banks - Regional              309
Software - Application        148
Medical Devices               104
Software - Infrastructure      90
                             ... 
Copper                          3
Financial Conglomerates         3
Infrastructure Operations       1
Uranium                         1
Telecom Services - Foreign      1
Name: sector, Length: 146, dtype: int64

In [ ]:
# saves for presentation

In [136]:
df[397:4601].to_csv('murica.csv', index=False)

In [77]:
df.to_csv('cleaned.csv', index=False)

In [ ]:
# wanted to create a nlp-model to get sector out of description but time ...

from googletrans import Translator

translator = Translator(service_urls=['translate.google.com'])

df['sector'] = np.where(df['lang_s']=='de', df['sector'].apply(lambda x: translator.translate(x, dest='en').text), df['sector'])
df


In [ ]:
import yake

##### stem and lem inbefore!!!

kw_extractor = yake.KeywordExtractor()
text = """
Die 7C Solarparken AG zählt zu den mittelgroßen Betreibern von Solarparks in Deutschland. Alle laufenden Tätigkeiten erfolgen mit Eigenkompetenzen der Gesellschaft. 7C Solarparken verfügt über eine eigene O&M-Abteilung. Sämtliche administrative und unterstützende kaufmännische Tätigkeiten in den Bereichen Akquise, Finanzierung und Verbesserung der laufenden Betreibergesellschaften werden hausintern durchgeführt. Die 7C Solarparken AG ist im September 2014 aus der Übernahme der Colexon Energy AG durch die belgische 7C Solarparken NV hervorgegangen.
"""
language = "de"
max_ngram_size = 1
deduplication_threshold = 0.3
numOfKeywords = 10
custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, top=numOfKeywords, features=None)
keywords = custom_kw_extractor.extract_keywords(text)

for kw in keywords:
    print(kw)

('Solarparken', 0.04993372328294843)
('Deutschland', 0.0676124936337362)
('Betreibern', 0.08504893984356532)
('zählt', 0.2141628356917364)
('mittelgroßen', 0.2141628356917364)
('Eigenkompetenzen', 0.24404926327940085)
('Akquise', 0.3101897629925119)
('Finanzierung', 0.3101897629925119)
('Übernahme', 0.3968605949782225)
('Colexon', 0.3968605949782225)
